# Trabalho Final de Plataformas cognitivas

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
import os

from google.cloud import bigquery

In [3]:
#@title Informe o id do projeto nesta linha e o bucket. Sugestão, coloque o nome do bucket como nome do projeto + um sufixo:
PROJECT = 'trabalho-final-plat-cog-fiap' #@param {type:"string"}
BUCKET = 'trabalho-final-plat-cog-fiap-bucket' #@param {type:"string"}

In [4]:
#Outras variáveis:
REGION = 'us-central1'

#Variáveis do OS
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

In [5]:
%%bash
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: trabalho-final-plat-cog-fiap


In [6]:
%%bash

# Cria um BigQuery dataset
datasetexists=$(bq ls -d --project_id=$PROJECT | grep -w analise_credito)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset já existe. Show, não vamos precisar criá-lo."

else
    echo "Criando BigQuery dataset chamado: analise_credito"
    
    bq --location=US mk --dataset \
        --description "Dados de Crédito Pessoal" \
        $PROJECT:analise_credito
fi
echo "Estes são os datasets do projeto:"
bq ls -d --project_id=$PROJECT   

BigQuery dataset já existe. Show, não vamos precisar criá-lo.
Estes são os datasets do projeto:
     datasetId     
 ----------------- 
  analise_credito  
  pesobebes        


In [7]:
%%bash
## Cria um GCS bucket se não existir already...
exists=$(gsutil ls -d -p ${PROJECT}  | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
    echo -e "Bucket já existe. Show, não vamos precisar criá-lo."
    
else
    echo "Criando um novo GCS bucket."
    gsutil mb -l ${REGION} -p ${PROJECT} gs://${BUCKET}
fi
echo "Estes são os buckets do projeto:"
gsutil ls -p ${PROJECT}

Bucket já existe. Show, não vamos precisar criá-lo.
Estes são os buckets do projeto:
gs://trabalho-final-plat-cog-fiap-bucket/


In [8]:
%%bigquery --project {PROJECT}
CREATE OR REPLACE TABLE
    analise_credito.analise_credito AS
SELECT
* ,
RAND() AS meurand
FROM
    emf-teacher.trabalho_loans.loan_default

Query is running:   0%|          |

""


In [9]:
%%bigquery --project {PROJECT}
CREATE OR REPLACE TABLE
    analise_credito.analise_credito_treino AS
SELECT
    *,
  FROM analise_credito.analise_credito
WHERE meurand < 0.7

Query is running:   0%|          |

""


In [10]:
%%bigquery --project {PROJECT}
CREATE OR REPLACE TABLE
    analise_credito.analise_credito_teste AS
SELECT
    *,
  FROM analise_credito.analise_credito
WHERE meurand >= 0.7

Query is running:   0%|          |

""


In [11]:
# Criamos um BigQuery client object.
bqclient=bigquery.Client(project=PROJECT)

In [12]:
query = """
SELECT
  *
FROM
  analise_credito.analise_credito_treino
LIMIT 100
"""
df_treino = bqclient.query(query).to_dataframe()
query = """
SELECT
  *
FROM
  analise_credito.analise_credito_teste
LIMIT 100
"""
df_teste = bqclient.query(query).to_dataframe()

In [13]:
dataset_name = "analise_credito"

# Criamos um objeto de referência para o dataset
dataset_ref = bqclient.dataset( dataset_id=dataset_name, project=bqclient.project)

# Export ambas tabelas train and eval tables
for step in ["treino", "teste"]:
    destination_uri = os.path.join(
        "gs://", BUCKET, dataset_name, "data", "{}*.csv".format(step))
    table_name = "analise_credito_{}".format(step)
    table_ref = dataset_ref.table(table_name)
    extract_job = bqclient.extract_table(
        table_ref,
        destination_uri,
        # Local do bucket deve ser o mesmo do dataset
        location="US",
    )  # API request
    extract_job.result()  # Espera a exportação acontecer.

    print("Exported {}:{}.{} to {}".format(
        bqclient.project, dataset_name, table_name, destination_uri))

Exported trabalho-final-plat-cog-fiap:analise_credito.analise_credito_treino to gs://trabalho-final-plat-cog-fiap-bucket/analise_credito/data/treino*.csv
Exported trabalho-final-plat-cog-fiap:analise_credito.analise_credito_teste to gs://trabalho-final-plat-cog-fiap-bucket/analise_credito/data/teste*.csv


In [14]:
%%bash
rm -rf ./data
mkdir ./data
gsutil cp gs://${BUCKET}/analise_credito/data/*.csv ./data/

Copying gs://trabalho-final-plat-cog-fiap-bucket/analise_credito/data/raw.csv...
Copying gs://trabalho-final-plat-cog-fiap-bucket/analise_credito/data/teste000000000000.csv...
Copying gs://trabalho-final-plat-cog-fiap-bucket/analise_credito/data/treino000000000000.csv...
| [3 files][ 54.6 MiB/ 54.6 MiB]                                                
Operation completed over 3 objects/54.6 MiB.                                     


In [15]:
%%bash
wc -l ./data/*.csv
head ./data/*.csv
tail ./data/*.csv

  148671 ./data/raw.csv
   44844 ./data/teste000000000000.csv
  103828 ./data/treino000000000000.csv
  297343 total
==> ./data/raw.csv <==
ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,Neg_ammortization,interest_only,lump_sum_payment,property_value,construction_type,occupancy_type,Secured_by,total_units,income,credit_type,Credit_Score,co_applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
26364,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,496500,,,,360,not_neg,not_int,not_lpsm,,sb,pr,home,1U,6600,EQUI,856,EXP,35-44,not_inst,,south,direct,1,
27402,2019,cf,Male,nopre,type1,p1,l1,nopc,nob/c,196500,,,,276,neg_amm,not_int,not_lpsm,,sb,pr,home,1U,4260,EQUI,512,EXP,35-44,to_inst,,North,direct,1,
28438,2019,ncf,Male,pre,type1,p1,l1,nopc,nob/c,246500,,,,360,not_neg,not_int,not_lpsm,,sb,pr,home,1U,2460,EQUI,503,EXP,55-64,not